**Step 1: Environment and Data Setup**

In [ ]:
# Initializing the necessary liabries and prompt the user to upload the data

import pandas as pd
import numpy as np
from google.colab import files
!pip install xlsxwriter -q


print("Please upload your file: ")
uploaded = files.upload()

Please upload your file: 


Saving Current_Market_Prices.xlsx to Current_Market_Prices.xlsx
Saving Supplier_A_Data.xlsx to Supplier_A_Data.xlsx
Saving Supplier_B_Data.xlsx to Supplier_B_Data.xlsx


**Step 2: Data Organization and Health Check**

In [ ]:
# Loading data into DataFrames
df_a = pd.read_excel('Supplier_A_Data.xlsx')
df_b = pd.read_excel('Supplier_B_Data.xlsx')
df_market = pd.read_excel('Current_Market_Prices.xlsx')

# Now we read the files into separate variables
df_supplier_a = pd.read_excel('Supplier_A_Data.xlsx')
df_supplier_b = pd.read_excel('Supplier_B_Data.xlsx')
df_market_prices = pd.read_excel('Current_Market_Prices.xlsx')

# --- DATA VERIFICATION (The Health Check) ---
print("\n--- Supplier A Preview ---")
print(df_supplier_a.head())

print("\n--- Supplier B Preview ---")
print(df_supplier_b.head())

print("\n--- Market Prices Preview ---")
print(df_market_prices.head())

# Check data types to ensure pricing columns are 'float' or 'int'
print("\n--- Column Data Types ---")
print(df_supplier_a.dtypes)

print("Data successfully loaded and standardized.")


--- Supplier A Preview ---
        SKU        Product_Name  Wholesale_Cost_A  Lead_Time_Days
0  SKU-1000     GigaMax Webcams            505.01              10
1  SKU-1001    GigaMax Monitors            161.02              14
2  SKU-1002    GigaMax Monitors            612.80              10
3  SKU-1003  GigaMax Headphones            833.01               5
4  SKU-1004       Apex Monitors            190.38              10

--- Supplier B Preview ---
     ItemID  Factory_Price  Shipping_Fee  Stock_Level
0  SKU-1000         516.18          7.57           71
1  SKU-1001         164.58          6.92          115
2  SKU-1002         626.35         39.89           51
3  SKU-1003         851.43          8.61          495
4  SKU-1004         194.59         15.43          301

--- Market Prices Preview ---
  Product_ID  Current_Retail_Price Last_Updated
0   SKU-1000                707.01   2026-01-25
1   SKU-1001                225.43   2026-01-25
2   SKU-1002                857.92   2026-01-25
3

**Step 3: Necessary Data Cleaning**

In [ ]:
# We create a map for Supplier B and Market Prices to match Supplier A's 'SKU'
# This ensures we have a common 'key' to join the files later.

mapping_supplier_b = {
    'ItemID': 'SKU',
    'Factory_Price': 'Wholesale_Cost_B'
}

mapping_market = {
    'Product_ID': 'SKU',
    'Current_Retail_Price': 'Retail_Price'
}

# --- STEP 2: APPLY THE RENAMING ---
# We use inplace=True to modify the dataframes directly
df_supplier_b.rename(columns=mapping_supplier_b, inplace=True)
df_market_prices.rename(columns=mapping_market, inplace=True)

print("\n--- Supplier A Preview ---")
print(df_supplier_a.head())

print("\n--- Supplier B Preview ---")
print(df_supplier_b.head())

print("\n--- Market Prices Preview ---")
print(df_market_prices.head())


--- Supplier A Preview ---
        SKU        Product_Name  Wholesale_Cost_A  Lead_Time_Days
0  SKU-1000     GigaMax Webcams            505.01              10
1  SKU-1001    GigaMax Monitors            161.02              14
2  SKU-1002    GigaMax Monitors            612.80              10
3  SKU-1003  GigaMax Headphones            833.01               5
4  SKU-1004       Apex Monitors            190.38              10

--- Supplier B Preview ---
        SKU  Wholesale_Cost_B  Shipping_Fee  Stock_Level
0  SKU-1000            516.18          7.57           71
1  SKU-1001            164.58          6.92          115
2  SKU-1002            626.35         39.89           51
3  SKU-1003            851.43          8.61          495
4  SKU-1004            194.59         15.43          301

--- Market Prices Preview ---
        SKU  Retail_Price Last_Updated
0  SKU-1000        707.01   2026-01-25
1  SKU-1001        225.43   2026-01-25
2  SKU-1002        857.92   2026-01-25
3  SKU-1003       1

**Step 4: Ensuring All Financial Columns are Numeric**

In [ ]:
# we ensure all financial columns are strictly numeric (float).
cols_to_fix = {
    'df_supplier_a': ['Wholesale_Cost_A'],
    'df_supplier_b': ['Wholesale_Cost_B', 'Shipping_Fee'],
    'df_market_prices': ['Retail_Price']
}

# Force conversion to numeric, turning errors into 'NaN' (Not a Number)
df_supplier_a['Wholesale_Cost_A'] = pd.to_numeric(df_supplier_a['Wholesale_Cost_A'], errors='coerce')
df_supplier_b['Wholesale_Cost_B'] = pd.to_numeric(df_supplier_b['Wholesale_Cost_B'], errors='coerce')
df_supplier_b['Shipping_Fee'] = pd.to_numeric(df_supplier_b['Shipping_Fee'], errors='coerce')
df_market_prices['Retail_Price'] = pd.to_numeric(df_market_prices['Retail_Price'], errors='coerce')

print("✅ Column names standardized and data types verified as numeric.")
print(f"Standardized Columns: {df_supplier_b.columns.tolist()}")

✅ Column names standardized and data types verified as numeric.
Standardized Columns: ['SKU', 'Wholesale_Cost_B', 'Shipping_Fee', 'Stock_Level']


**Step 5: The Logic Engine**

In [ ]:
# We take the SKU and Name from our most reliable source (Supplier A)
df_report = df_supplier_a[['SKU', 'Product_Name']].copy()

# We use 'map' or 'merge' locally to fetch specific columns
df_report = df_report.merge(df_supplier_a[['SKU', 'Wholesale_Cost_A']], on='SKU', how='left')

# Mapping Supplier B's data and calculating their specific "Landed Cost"
df_report = df_report.merge(df_supplier_b[['SKU', 'Wholesale_Cost_B', 'Shipping_Fee']], on='SKU', how='left')
df_report['Landed_Cost_B'] = df_report['Wholesale_Cost_B'] + df_report['Shipping_Fee']

# Fetching the Retail Price
df_report = df_report.merge(df_market_prices[['SKU', 'Retail_Price']], on='SKU', how='left')

# Now we compare (Wholesale_Cost_A) vs (Landed_Cost_B)
df_report['Best_Cost'] = df_report[['Wholesale_Cost_A', 'Landed_Cost_B']].min(axis=1)

# Preview the result
print("--- Logic Engine Report Preview ---")
print(df_report[['SKU', 'Wholesale_Cost_A', 'Landed_Cost_B', 'Best_Cost', 'Retail_Price']].head())

--- Logic Engine Report Preview ---
        SKU  Wholesale_Cost_A  Landed_Cost_B  Best_Cost  Retail_Price
0  SKU-1000            505.01         523.75     505.01        707.01
1  SKU-1001            161.02         171.50     161.02        225.43
2  SKU-1002            612.80         666.24     612.80        857.92
3  SKU-1003            833.01         860.04     833.01       1166.21
4  SKU-1004            190.38         210.02     190.38        266.53


Step 6: Calculating the Profit Margin

In [ ]:
# Formula: (Retail Price - Best Cost) / Retail Price
df_report['Margin_Pct'] = (df_report['Retail_Price'] - df_report['Best_Cost']) / df_report['Retail_Price']

# We flag any product with a margin below 20% as 'REPRICE'
df_report['Action_Status'] = df_report['Margin_Pct'].apply(
    lambda x: 'REPRICE' if x < 0.2857 else 'OK'
)

# Preview the analytical results
print("--- Final Analysis Preview ---")
print(df_report[['SKU', 'Best_Cost', 'Retail_Price', 'Margin_Pct', 'Action_Status']].head())

--- Final Analysis Preview ---
        SKU  Best_Cost  Retail_Price  Margin_Pct Action_Status
0  SKU-1000     505.01        707.01    0.285710            OK
1  SKU-1001     161.02        225.43    0.285721            OK
2  SKU-1002     612.80        857.92    0.285714            OK
3  SKU-1003     833.01       1166.21    0.285712            OK
4  SKU-1004     190.38        266.53    0.285709            OK


**Step 7: Formatting and Exporting the Script To Excel**

In [ ]:
# 1: INITIALIZE THE WRITER ---
file_name = 'Pricing_Analysis_Report.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

# Send the 10-column dataframe to Excel
df_report.to_excel(writer, index=False, sheet_name='Pricing_Analysis')

workbook  = writer.book
worksheet = writer.sheets['Pricing_Analysis']

# DEFINE FORMATS ---
header_fmt   = workbook.add_format({'bold': True, 'bg_color': '#1F4E78', 'font_color': 'white', 'border': 1})
currency_fmt = workbook.add_format({'num_format': '$#,##0.00', 'border': 1})
percent_fmt  = workbook.add_format({'num_format': '0.00000%', 'border': 1})
standard_fmt = workbook.add_format({'border': 1})
warning_fmt  = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006', 'border': 1})

# PRECISE COLUMN INDEX MAPPING (0 to 9)
worksheet.set_column(0, 1, 20, standard_fmt)
worksheet.set_column(2, 7, 15, currency_fmt)
worksheet.set_column(8, 8, 15, percent_fmt)
worksheet.set_column(9, 9, 15, standard_fmt)

# HEADERS & VISUALS
for col_num, value in enumerate(df_report.columns.values):
    worksheet.write(0, col_num, value, header_fmt)

# Add Data Bars to the Margin column (Col 8 / H)
worksheet.conditional_format('I2:I500', {
    'type': 'data_bar',
    'bar_color': '#63BE7B'
})

# Highlight 'REPRICE' in Red (Col 9 / J)
worksheet.conditional_format('J2:J500', {
    'type':     'cell',
    'criteria': '==',
    'value':    '"REPRICE"',
    'format':   warning_fmt
})

# SAVE
writer.close()
files.download(file_name)

print("✅ Formatting Done and the Excel File Has been Exported")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Formatting Done and the Excel File Has been Exported
